<a href="https://colab.research.google.com/github/kyle-gc/NeuralNet/blob/main/Kick_Starter_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

import pandas as pd
url = 'https://raw.githubusercontent.com/kyle-gc/NeuralNet/main/DataTrain.csv'
kick = pd.read_csv(url)
data = np.array(kick)

X, y = data[:, :-1], data[:, -1]

scaler = MinMaxScaler([0, 1])
X = scaler.fit_transform(X)


X_total, y_total = data[:, :-1], data[:, -1]

c0 = 0
c1 = 1
sub_index = np.logical_or(y_total == c0, y_total == c1)
X = X_total[sub_index]
y = y_total[sub_index]
y[y==c0] = 0
y[y==c1] = 1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=123)


In [2]:
#Sigmoid Function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#Class Neural Net
class NeuralNet:
#Initial Parameters
    def __init__(self):
        rng = np.random.RandomState(42)
        self.W = rng.randn(8, 8)
        self.B1 = np.zeros(8)
        self.U = rng.randn(8, 1)
        self.B2 = np.zeros(1)

#Feed Forward Algorithm
    def forward(self, X):
        self.X = X
        self.A = np.dot(X, self.W) + self.B1
        self.H = sigmoid(self.A)
        self.B = np.dot(self.H, self.U) + self.B2
        self.P = sigmoid(self.B)
        return self.P.squeeze()

#Backwards Propagation
    def backwards(self, dP):
        dB = (1 - self.P) * self.P * dP
        dB2 = np.sum(dB, axis=0)
        dU = np.dot(self.H.T, dB)
        dH = np.dot(dB, self.U.T)
        dA = (1 - self.H) * self.H * dH
        dB1 = np.sum(dA, axis=0)
        dW = np.dot(self.X.T, dA)
        self.dU = dU
        self.dW = dW
        self.dB1 = dB1
        self.dB2 = dB2
        

#Loss function
def loss(pred, y):
    return - (y * np.log(pred) + (1 - y) * np.log(1 - pred)).mean()

#Derrivative of the Loss Function
def dloss(pred, y):
    return - (y / pred - (1 - y ) / (1 - pred)) / pred.size


In [36]:
nn = NeuralNet()
learning_rate = 0.6
MAX_ITER = 30000
iter_counter = 0

while True:
    pred = nn.forward(X_train)
    loss_value = loss(pred, y_train)
    dpred = dloss(pred, y_train)[:, np.newaxis]
    nn.backwards(dpred)

# Momentum factor added to the model to train faster

    #delt1 = -1 * nn.dW * learning_rate
    nn.W -= nn.dW * learning_rate #+ (0.4 * delt1)

    #delt2 = -1 * nn.dU * learning_rate
    nn.U -= nn.dU * learning_rate #+ (0.4 * delt2)

    #delt3 = -1 * nn.dB1 * learning_rate
    nn.B1 -= nn.dB1 * learning_rate #+ (0.4 * delt3)
    
    #delt4 = -1 * nn.dB2 * learning_rate
    nn.B2 -= nn.dB2 * learning_rate #+ (0.4 * delt4)

    iter_counter += 1
    if iter_counter % 100 == 0:
        print(iter_counter, loss_value)

    if iter_counter >= MAX_ITER:
        break


100 0.6324771457127202
200 0.6212810163069392
300 0.6097581987167543
400 0.5972289165021413
500 0.5838130535722409
600 0.5699766507372257
700 0.5562122111406845
800 0.5428180450753275
900 0.5299084937610229
1000 0.5175078529086896
1100 0.5056128648176935
1200 0.49421634137698633
1300 0.4833117969731374
1400 0.4728923759668144
1500 0.46294925329586795
1600 0.45347073437851726
1700 0.44444208842781885
1800 0.435845928969083
1900 0.4276629129895685
2000 0.41987253150096376
2100 0.41245382506208095
2200 0.40538594732494837
2300 0.3986486784433805
2400 0.5020327920843718
2500 0.45807021695045036
2600 0.44762639261694276
2700 0.4438654197927433
2800 0.44094146291506175
2900 0.43820513918030596
3000 0.4355856202306833
3100 0.4330661093877144
3200 0.4306366152990977
3300 0.4282893864771938
3400 0.4260180249417355
3500 0.4238170767518597
3600 0.42168178343546003
3700 0.4196079214024155
3800 0.41759169438658833
3900 0.41562965890643583
4000 0.41371867077955926
4100 0.4118558453839325
4200 0.4100

In [37]:
predict = nn.forward(X_test)
loss_value_test = loss(predict, y_test)
print(loss_value_test)

0.2857846500742674
